In [4]:
# ✅ 라이브러리 설치
!pip install folium

# ✅ Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 기본 라이브러리
import pandas as pd
import folium
from folium.plugins import HeatMap

# ✅ CSV 경로 (본인의 Google Drive 경로에 맞게 수정)
logs_path = '/content/drive/MyDrive/seoul-metro-2021.logs.csv'
station_path = '/content/drive/MyDrive/seoul-metro-station-info.csv'

# ✅ 데이터 불러오기
logs_df = pd.read_csv(logs_path)
station_df = pd.read_csv(station_path)

# ✅ timestamp 컬럼 datetime 변환
logs_df['timestamp'] = pd.to_datetime(logs_df['timestamp'], errors='coerce')

# ✅ 출근 시간 (09시 이전), 퇴근 시간 (17시 이후) 추출
morning_df = logs_df[logs_df['timestamp'].dt.hour < 9]
evening_df = logs_df[logs_df['timestamp'].dt.hour >= 17]

# ✅ 그룹화 (역 코드 기준 합계 계산)
morning_grouped = morning_df.groupby('station_code')[['people_in', 'people_out']].sum()
evening_grouped = evening_df.groupby('station_code')[['people_in', 'people_out']].sum()

# ✅ 지하철역 위치 정보 정리
station_info = station_df[['station.code', 'geo.latitude', 'geo.longitude']]
station_info.columns = ['station_code', 'latitude', 'longitude']
station_info.set_index('station_code', inplace=True)

# ✅ 위치정보 병합
morning_merged = morning_grouped.join(station_info, how='inner')
evening_merged = evening_grouped.join(station_info, how='inner')

# ✅ 히트맵 시각화 - 출근
m1 = folium.Map(location=[37.5665, 126.9780], zoom_start=11)
HeatMap(data=morning_merged[['latitude', 'longitude', 'people_out']].dropna().values,
        radius=10).add_to(m1)
m1.save('morning_heatmap.html')

# ✅ 히트맵 시각화 - 퇴근
m2 = folium.Map(location=[37.5665, 126.9780], zoom_start=11)
HeatMap(data=evening_merged[['latitude', 'longitude', 'people_in']].dropna().values,
        radius=10).add_to(m2)
m2.save('evening_heatmap.html')

# ✅ 유동인구 많은 상위 지하철역 Top3 출력
top_morning = morning_grouped['people_out'].sort_values(ascending=False).head(3)
top_evening = evening_grouped['people_in'].sort_values(ascending=False).head(3)

print("🚉 출근 시간 하차 인원 Top 3 역:")
print(top_morning)

print("\n🚇 퇴근 시간 승차 인원 Top 3 역:")
print(top_evening)

m2


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚉 출근 시간 하차 인원 Top 3 역:
station_code
2748    6067855
222     5757499
221     5521983
Name: people_out, dtype: int64

🚇 퇴근 시간 승차 인원 Top 3 역:
station_code
222    14845284
219     9535958
239     9453758
Name: people_in, dtype: int64
